[View in Colaboratory](https://colab.research.google.com/github/gowtham91m/Predicting-IDC-in-Breast-Cancer-Histology-Images/blob/master/IDC.ipynb)

In [1]:
# Install Kaggle API for download competition data
!pip3 install -q kaggle
import os,shutil,fnmatch, random
from glob import glob
from time import time
import cv2
import keras
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json, Model
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
os.chdir('/content')
if 'kaggle.txt' in ('/content'): os.remove('kaggle.text')
from google.colab import files
downloaded = files.upload()

Saving kaggle.txt to kaggle (5).txt


In [3]:
with open('kaggle.txt') as f:
    key = f.read()
os.environ['KAGGLE_USERNAME']="gowham91m"
os.environ['KAGGLE_KEY']=key
 
!kaggle datasets download -d paultimothymooney/breast-histopathology-images

breast-histopathology-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!unzip -q -o breast-histopathology-images.zip -d breast-histopathology-images
os.chdir('/content/breast-histopathology-images')
!unzip -q -o IDC_regular_ps50_idx5.zip -d IDC_regular_ps50_idx5
os.chdir('/content/breast-histopathology-images/IDC_regular_ps50_idx5')

In [5]:
negative_class_images = glob('/content/breast-histopathology-images/IDC_regular_ps50_idx5/**/0/*.png', recursive=True)
print(len(negative_class_images))

positive_class_images = glob('/content/breast-histopathology-images/IDC_regular_ps50_idx5/**/1/*.png', recursive=True)
print(len(positive_class_images))

198738
78786


In [0]:
if 'IDC' in os.listdir('/content'): shutil.rmtree('/content/IDC')
if 'IDC' not in os.listdir('/content'): os.mkdir('/content/IDC')
  
if 'train' not in os.listdir('/content/IDC'): os.mkdir('/content/IDC/train')
if '0' not in os.listdir('/content/IDC/train'): os.mkdir('/content/IDC/train/0')
if '1' not in os.listdir('/content/IDC/train'): os.mkdir('/content/IDC/train/1')
  
if 'val' not in os.listdir('/content/IDC'): os.mkdir('/content/IDC/val')
if '0' not in os.listdir('/content/IDC/val'): os.mkdir('/content/IDC/val/0')
if '1' not in os.listdir('/content/IDC/val'): os.mkdir('/content/IDC/val/1')

In [0]:
for file in glob('/content/breast-histopathology-images/IDC_regular_ps50_idx5/**/0/*.png'):
    shutil.copy2(file, '/content/IDC/train/0');
for file in glob('/content/breast-histopathology-images/IDC_regular_ps50_idx5/**/1/*.png'):
    shutil.copy2(file, '/content/IDC/train/1');

In [0]:
train_neg_path = '/content/IDC/train/0'
train_pos_path = '/content/IDC/train/1'

val_neg_path = '/content/IDC/val/0'
val_pos_path = '/content/IDC/val/1'

# split train date into train and validation
train_neg_len = len(os.listdir('/content/IDC/train/0'))
val_neg_len = train_neg_len * 0.3

train_pos_len = len(os.listdir('/content/IDC/train/1'))
val_pos_len = train_pos_len * 0.3

val_pos = random.sample(os.listdir(train_pos_path),int(val_pos_len))
val_neg = random.sample(os.listdir(train_neg_path),int(val_neg_len))

In [0]:
for file in val_pos:
  try: shutil.move(os.path.join(train_pos_path,file), val_pos_path)
  except: pass
for file in val_neg:
  try: shutil.move(os.path.join(train_neg_path,file), val_neg_path)
  except: pass

In [10]:
print(len(os.listdir('/content/IDC/train/0')))
print(len(os.listdir('/content/IDC/train/1')))

print(len(os.listdir('/content/IDC/val/0')))
print(len(os.listdir('/content/IDC/val/1')))

139117
55151
59621
23635


In [11]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

batch_size=256
train_generator = train_datagen.flow_from_directory(
        '/content/IDC/train',  # this is the target directory
        target_size=(50, 50),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 


validation_generator = val_datagen.flow_from_directory(
        '/content/IDC/val',
        target_size=(50, 50),
        batch_size=batch_size,
        class_mode='binary')

Found 194268 images belonging to 2 classes.
Found 83256 images belonging to 2 classes.


# Training CNN from scratch

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=( 50, 50, 3)))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

start_time = time()
model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        callbacks=[ModelCheckpoint('CNN.model', monitor='val_acc', save_best_only=True)])
model.save_weights('cnn.h5')

print('time taken ',time()-start_time)

Epoch 1/16
759/759 [==============================] - 304s 400ms/step - loss: 0.4176 - acc: 0.8191 - val_loss: 0.3758 - val_acc: 0.8377
Epoch 2/16
759/759 [==============================] - 284s 375ms/step - loss: 0.3763 - acc: 0.8404 - val_loss: 0.3458 - val_acc: 0.8523
Epoch 3/16
759/759 [==============================] - 282s 371ms/step - loss: 0.3612 - acc: 0.8485 - val_loss: 0.3488 - val_acc: 0.8548
Epoch 4/16
759/759 [==============================] - 277s 364ms/step - loss: 0.3484 - acc: 0.8530 - val_loss: 0.3588 - val_acc: 0.8502
Epoch 5/16
759/759 [==============================] - 277s 365ms/step - loss: 0.3395 - acc: 0.8576 - val_loss: 0.3199 - val_acc: 0.8628
Epoch 6/16
759/759 [==============================] - 276s 364ms/step - loss: 0.3335 - acc: 0.8596 - val_loss: 0.3217 - val_acc: 0.8641
Epoch 7/16
759/759 [==============================] - 274s 361ms/step - loss: 0.3304 - acc: 0.8625 - val_loss: 0.3304 - val_acc: 0.8621
Epoch 8/16
759/759 [============================

# Using pre training VGG16 model

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input

In [0]:
img_rows, img_cols, img_channel = 50, 50, 3
base_model = VGG16(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

In [0]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

In [0]:
vgg_model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
vgg_model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
vgg_model.fit_generator(
        train_generator,
        epochs=16,
        validation_data=validation_generator,
        callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)])
vgg_model.save_weights('vgg16.h5')

print('time taken ',time()-start_time)